In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COVID-19 mRNA Vaccine Degradation/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COVID-19 mRNA Vaccine Degradation/test.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130560 entries, 0 to 130559
Data columns (total 45 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      130560 non-null  object 
 1   id_seqpos               130560 non-null  object 
 2   sequence                130560 non-null  object 
 3   structure               130560 non-null  object 
 4   predicted_loop_type     130560 non-null  object 
 5   reactivity              130560 non-null  float64
 6   reactivity_error        130560 non-null  float64
 7   deg_Mg_pH10             130560 non-null  float64
 8   deg_error_Mg_pH10       130560 non-null  float64
 9   deg_pH10                130560 non-null  float64
 10  deg_error_pH10          130560 non-null  float64
 11  deg_Mg_50C              130560 non-null  float64
 12  deg_error_Mg_50C        130560 non-null  float64
 13  deg_50C                 130560 non-null  float64
 14  deg_error_50C       

In [ ]:
train.isna().sum().sum()

0

In [ ]:
train.describe()

,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,deg_error_pH10,deg_Mg_50C,deg_error_Mg_50C,deg_50C,deg_error_50C
count,130560.000000,130560.000000,130560.00000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000,130560.000000
mean,0.376570,2825.710861,0.44650,2332.806124,0.448538,6288.673592,0.405686,3579.976118,0.427428,5969.595514
std,0.730013,19105.915817,0.70582,14614.719469,1.300353,42323.269166,0.840844,23229.704416,1.126904,38182.352844
min,-20.664800,0.001900,-14.74050,0.006100,-44.515300,0.008300,-24.252500,0.004100,-38.339900,0.006500
25%,0.043800,0.044900,0.13130,0.057200,0.099900,0.073900,0.074100,0.052000,0.092700,0.081300
50%,0.216200,0.083000,0.28990,0.098800,0.276100,0.120500,0.256800,0.088800,0.308800,0.133700
75%,0.540400,0.172500,0.56760,0.200000,0.569100,0.244200,0.560000,0.181600,0.613825,0.265325
max,20.667500,146151.225000,14.74010,104235.174200,44.521200,314833.569500,24.252500,171525.321700,38.343800,271158.604000


In [ ]:
train.head()

,id,id_seqpos,sequence,structure,predicted_loop_type,reactivity,reactivity_error,deg_Mg_pH10,deg_error_Mg_pH10,deg_pH10,...,b4_structure,a4_structure,b4_predicted_loop_type,a4_predicted_loop_type,b5_sequence,a5_sequence,b5_structure,a5_structure,b5_predicted_loop_type,a5_predicted_loop_type
0,id_159f0ff46,id_159f0ff46_44,A,.,H,1.7786,0.1195,0.7346,0.0790,0.3680,...,(,),S,S,C,A,.,),I,S
1,id_d73546f4e,id_d73546f4e_38,G,.,I,0.4397,0.1033,0.1630,0.0799,0.0878,...,(,.,S,H,C,C,.,.,B,H
2,id_f3037fedf,id_f3037fedf_34,G,(,S,0.1120,0.1375,0.0000,0.0744,0.1835,...,(,(,S,S,A,G,(,(,S,S
3,id_468300749,id_468300749_12,G,(,S,0.0622,0.0471,0.1179,0.1075,0.0657,...,.,(,E,S,A,U,.,(,E,S
4,id_69e3878a2,id_69e3878a2_28,U,(,S,0.1378,0.1352,0.4069,0.2168,0.3375,...,(,(,S,S,G,G,(,.,S,H


In [ ]:
print(train.iloc[1,:].to_list())

['id_d73546f4e', 'id_d73546f4e_38', 'G', '.', 'I', 0.4397, 0.1033, 0.163, 0.0799, 0.0878, 0.0878, 0.11, 0.0683, 0.0666, 0.0864, 'G', 'A', '(', '.', 'S', 'I', 'C', 'C', '(', '(', 'S', 'S', 'A', 'G', '(', '(', 'S', 'S', 'G', 'U', '(', '.', 'S', 'H', 'C', 'C', '.', '.', 'B', 'H']


In [ ]:
print(train[train.id =='id_159f0ff46'])

                  id        id_seqpos sequence structure predicted_loop_type  \
0       id_159f0ff46  id_159f0ff46_44        A         .                   H   
334     id_159f0ff46   id_159f0ff46_5        C         (                   S   
525     id_159f0ff46   id_159f0ff46_4        A         .                   E   
6303    id_159f0ff46  id_159f0ff46_50        C         )                   S   
6794    id_159f0ff46  id_159f0ff46_53        A         )                   S   
7254    id_159f0ff46  id_159f0ff46_58        U         )                   S   
8589    id_159f0ff46   id_159f0ff46_3        A         .                   E   
9925    id_159f0ff46  id_159f0ff46_14        G         .                   H   
16120   id_159f0ff46  id_159f0ff46_62        U         )                   S   
17667   id_159f0ff46  id_159f0ff46_66        U         .                   M   
20429   id_159f0ff46  id_159f0ff46_30        U         (                   S   
22314   id_159f0ff46  id_159f0ff46_41   

In [ ]:
train.columns

Index(['id', 'id_seqpos', 'sequence', 'structure', 'predicted_loop_type',
       'reactivity', 'reactivity_error', 'deg_Mg_pH10', 'deg_error_Mg_pH10',
       'deg_pH10', 'deg_error_pH10', 'deg_Mg_50C', 'deg_error_Mg_50C',
       'deg_50C', 'deg_error_50C', 'b1_sequence', 'a1_sequence',
       'b1_structure', 'a1_structure', 'b1_predicted_loop_type',
       'a1_predicted_loop_type', 'b2_sequence', 'a2_sequence', 'b2_structure',
       'a2_structure', 'b2_predicted_loop_type', 'a2_predicted_loop_type',
       'b3_sequence', 'a3_sequence', 'b3_structure', 'a3_structure',
       'b3_predicted_loop_type', 'a3_predicted_loop_type', 'b4_sequence',
       'a4_sequence', 'b4_structure', 'a4_structure', 'b4_predicted_loop_type',
       'a4_predicted_loop_type', 'b5_sequence', 'a5_sequence', 'b5_structure',
       'a5_structure', 'b5_predicted_loop_type', 'a5_predicted_loop_type'],
      dtype='object')

In [ ]:
Q1 = np.percentile(train['reactivity_error'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['reactivity_error'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['reactivity_error'] = train['reactivity_error'].where((train['reactivity_error'] >= lower_bound) & (train['reactivity_error'] <= upper_bound), np.mean(train['reactivity_error']))
Q1 = np.percentile(train['deg_error_Mg_pH10'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_error_Mg_pH10'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_error_Mg_pH10'] = train['deg_error_Mg_pH10'].where((train['deg_error_Mg_pH10'] >= lower_bound) & (train['deg_error_Mg_pH10'] <= upper_bound), np.mean(train['deg_error_Mg_pH10']))
Q1 = np.percentile(train['deg_pH10'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_pH10'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_pH10'] = train['deg_pH10'].where((train['deg_pH10'] >= lower_bound) & (train['deg_pH10'] <= upper_bound), np.mean(train['deg_pH10']))
Q1 = np.percentile(train['deg_error_pH10'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_error_pH10'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_error_pH10'] = train['deg_error_pH10'].where((train['deg_error_pH10'] >= lower_bound) & (train['deg_error_pH10'] <= upper_bound), np.mean(train['deg_error_pH10']))

Q1 = np.percentile(train['deg_error_Mg_50C'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_error_Mg_50C'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_error_Mg_50C'] = train['deg_error_Mg_50C'].where((train['deg_error_Mg_50C'] >= lower_bound) & (train['deg_error_Mg_50C'] <= upper_bound), np.mean(train['deg_error_Mg_50C']))
Q1 = np.percentile(train['deg_50C'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_50C'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_50C'] = train['deg_50C'].where((train['deg_50C'] >= lower_bound) & (train['deg_50C'] <= upper_bound), np.mean(train['deg_50C']))
Q1 = np.percentile(train['deg_error_50C'], 25, interpolation = 'midpoint')
Q3 = np.percentile(train['deg_error_50C'], 75, interpolation = 'midpoint')
IQR = Q3 - Q1

upper_bound = Q3 + 1.5 * IQR
lower_bound = Q1 - 1.5 * IQR
#replace outliers with the mean of 'EXPERIENCE_LENGTH' col
train['deg_error_50C'] = train['deg_error_50C'].where((train['deg_error_50C'] >= lower_bound) & (train['deg_error_50C'] <= upper_bound), np.mean(train['deg_error_50C']))


In [ ]:
train.drop(['reactivity_error','deg_error_Mg_pH10','deg_error_pH10','deg_error_Mg_50C','deg_error_50C'], axis=1, inplace=True)


In [ ]:
obj_cols = train.select_dtypes(include='object').columns
feature_scale=[feature for feature in train.columns if feature not in ['reactivity','deg_Mg_pH10','deg_Mg_50C'] and feature not in obj_cols]
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(train[feature_scale])

MinMaxScaler()

In [ ]:
y=train[['reactivity','deg_Mg_pH10','deg_Mg_50C']]
## drop dependent feature from dataset
X=train.drop(['reactivity','deg_Mg_pH10','deg_Mg_50C'],axis=1)

# Use apply() to display value counts for each object column

le = LabelEncoder()

for feature in obj_cols:
    X[feature] = le.fit_transform(X[feature])

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor()

In [ ]:
model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
pred = model.predict(x_val)
rmse = sqrt(mean_squared_error(y_val, pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 0.6642864706553351


In [ ]:
from sklearn.metrics import SCORERS
print()